In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, cv
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV

In [2]:
print("Data loading....")
df = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/MachineLearning/master/dataset/Company%20Bankruptcy%20Prediction/data.csv")
print("Data load complete")

Data loading....
Data load complete


In [3]:
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),...,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,1.256969e-04,0.0,0.458143,0.000725,0.0,0.147950,0.147950,0.147950,0.169141,0.311664,0.017560,0.095921,0.138736,0.022102,0.848195,0.688979,0.688979,0.217535,4.980000e+09,0.000327,0.263100,0.363725,0.002259,0.001208,0.629951,0.021266,0.207576,0.792424,0.005024,...,0.190643,0.004094,0.001997,1.473360e-04,0.147308,0.334015,0.276920,0.001036,0.676269,0.721275,0.339077,0.025592,0.903225,0.002022,0.064856,7.010000e+08,6.550000e+09,0.593831,4.580000e+08,0.671568,0.424206,0.676269,0.339077,0.126549,0.637555,0.458609,0.520382,0.312905,0.118250,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,2.897851e-04,0.0,0.461867,0.000647,0.0,0.182251,0.182251,0.182251,0.208944,0.318137,0.021144,0.093722,0.169918,0.022080,0.848088,0.689693,0.689702,0.217620,6.110000e+09,0.000443,0.264516,0.376709,0.006016,0.004039,0.635172,0.012502,0.171176,0.828824,0.005059,...,0.182419,0.014948,0.004136,1.383910e-03,0.056963,0.341106,0.289642,0.005210,0.308589,0.731975,0.329740,0.023947,0.931065,0.002226,0.025516,1.065198e-04,7.700000e+09,0.593916,2.490000e+09,0.671570,0.468828,0.308589,0.329740,0.120916,0.641100,0.459001,0.567101,0.314163,0.047775,0,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,0.780284,2.361297e-04,25500000.0,0.458521,0.000790,0.0,0.177911,0.177911,0.193713,0.180581,0.307102,0.005944,0.092338,0.142803,0.022760,0.848094,0.689463,0.689470,0.217601,7.280000e+09,0.000396,0.264184,0.368913,0.011543,0.005348,0.629631,0.021248,0.207516,0.792484,0.005100,..

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

In [5]:
X_df = df.iloc[:, 1:]
y_df = df.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=123, stratify=y_df)
xgb = XGBClassifier(n_estimators=20, learning_rate=0.1, max_depth=50, use_label_encoder=False)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
Acc_score = accuracy_score(pred, y_test)
F1_score = f1_score(pred, y_test)
Precision_score = precision_score(pred, y_test)
print("Acc score : {:.2f}".format(Acc_score))
print("F1 score : {:.2f}".format(F1_score))
print("Precision score : {:.2f}".format(Precision_score))

Acc score : 0.97
F1 score : 0.29
Precision score : 0.18


In [6]:
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=123)
xgb = XGBClassifier(n_estimators=20, learning_rate=0.1, max_depth=50, use_label_encoder=False,
                   objective = "binary:hinge")

for train_idx, test_idx in cv.split(X_df, y_df):
    x_train, x_test = X_df.iloc[train_idx, :], X_df.iloc[test_idx, :]
    y_train, y_test = y_df[train_idx], y_df[test_idx]
    
    xgb.fit(x_train, y_train)
    pred = xgb.predict(x_test)
    Acc_score = accuracy_score(pred, y_test)
    F1_score = f1_score(pred, y_test)
    Precision_score = precision_score(pred, y_test)
    print("Acc score : {:.2f}".format(Acc_score))
    print("F1 score : {:.2f}".format(F1_score))
    print("Precision score : {:.2f}".format(Precision_score))

Acc score : 0.96
F1 score : 0.32
Precision score : 0.27
Acc score : 0.96
F1 score : 0.30
Precision score : 0.24
Acc score : 0.96
F1 score : 0.40
Precision score : 0.40
Acc score : 0.97
F1 score : 0.44
Precision score : 0.40


In [7]:
xgb = XGBClassifier()
# use_label_encoder=False, objective="binary:hinge"

xgb_param_grid = {'max_depth': [10,20], 'n_estimators': [10,20], 'learning_rate': [0.1],
                 'use_label_encoder' : [False], 'objective' : ["binary:hinge"]}

hr_grid = GridSearchCV(estimator=xgb,
                       param_grid=xgb_param_grid,
                       scoring='roc_auc',
                       n_jobs=8,
                       cv=5,
                       refit=True, 
                       return_train_score=True)

hr_grid.fit(X_df, y_df)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=8,
             param_grid={'learning_rate': [0.1], 'max_depth': [10, 20],
                         'n_estimators': [10, 20],
                         'objective': ['binary:hinge'],
 

In [8]:
hr_grid_df = pd.DataFrame(hr_grid.cv_results_)
hr_grid_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_objective,param_use_label_encoder,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,4.168770,0.188632,0.045109,0.009372,0.1,10,10,binary:hinge,False,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.718182,0.614394,0.655303,0.673485,0.597345,0.651742,0.043020,2,0.943182,0.931629,0.943182,0.937500,0.923106,0.935720,0.007621
1,11.716187,0.349817,0.056501,0.006845,0.1,10,20,binary:hinge,False,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.666288,0.599621,0.638258,0.653788,0.587118,0.629015,0.030685,4,0.948864,0.954545,0.951705,0.948864,0.931818,0.947159,0.007955
2,7.536730,0.510143,0.040024,0.009480,0.1,20,10,binary:hinge,False,"{'learning_rate': 0.1, 'max_depth': 20, 'n_est...",0.694318,0.636364,0.667424,0.684470,0.608329,0.658181,0.031760,1,0.951705,0.963068,0.948864,0.951705,0.940341,0.951136,0.007276
3,8.788274,1.607156,0.018734,0.007895,0.1,20,20,binary:hinge,False,"{'learning_rate': 0.1, 'max_depth': 20, 'n_est...",0.687500,0.629545,0.649621,0.664773,0.575755,0.641439,0.037918,3,0.963068,0.965909,0.960227,0.971591,0.957386,0.963636,0.004888


In [9]:
hr_grid_df[hr_grid_df['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_objective,param_use_label_encoder,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
2,7.53673,0.510143,0.040024,0.00948,0.1,20,10,binary:hinge,False,"{'learning_rate': 0.1, 'max_depth': 20, 'n_est...",0.694318,0.636364,0.667424,0.68447,0.608329,0.658181,0.03176,1,0.951705,0.963068,0.948864,0.951705,0.940341,0.951136,0.007276
